<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifiers/CONDITIONAL/CPU_CONDITIONAL_ClimateBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [2]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 15:17:56] [setup] RAM Tracking...
[codecarbon INFO @ 15:17:56] [setup] GPU Tracking...
[codecarbon INFO @ 15:17:56] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:17:56] [setup] CPU Tracking...
[codecarbon WARNING @ 15:17:56] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 15:17:58] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 15:17:58] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 15:17:58] >>> Tracker's metadata:
[codecarbon INFO @ 15:17:58]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 15:17:58]   Python version: 3.10.12
[codecarbon INFO @ 15:17:58]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 15:17:58]   Available RAM : 12.675 GB
[codecarbon INFO @ 15:17:58]   CPU count: 2
[codecarbon INFO @ 15:17:58]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 15:17:58]   GPU c

In [3]:
from huggingface_hub import notebook_login
notebook_login()

[codecarbon INFO @ 15:19:13] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 15:19:13] Energy consumed for all GPUs : 0.000249 kWh. Total GPU Power : 11.918089010207128 W
[codecarbon INFO @ 15:19:13] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:19:13] 0.001234 kWh of electricity used since the beginning.


In [4]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Manual Inputs


In [5]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
train_file="policy_classification_train.json"
test_file="policy_classification_test.json"


### Define labels ##################
label_names= ['UnconditionalLabel','ConditionalLabel']


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "climatebert/distilroberta-base-climate-f"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "CONDITIONAL-multilabel-climatebert"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=6.03e-5   #4.04e-5
batch_size = 16
num_train_epochs= 6
weight_decay=0.05
warmup_steps = 300
gradient_accumulation_steps = 1

# Automated

## Dataset

In [6]:
# Load Dataset from Hugging Face
data_files = {"train": train_file, "test": test_file}
dataset = load_dataset(repo_id, data_files=data_files)

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = dataset['train'].to_pandas()
test_df =dataset['test'].to_pandas()

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

[codecarbon INFO @ 15:19:28] Energy consumed for RAM : 0.000119 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 15:19:28] Energy consumed for all GPUs : 0.000299 kWh. Total GPU Power : 11.923102556208178 W
[codecarbon INFO @ 15:19:28] Energy consumed for all CPUs : 0.001063 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:19:28] 0.001480 kWh of electricity used since the beginning.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{0: 'ConditionalLabel', 1: 'UnconditionalLabel'}


In [7]:
# # given the high imbalance in positive and negative examples, we downsample the
# # negative examples
data = train_df.copy()
data['check'] = data.apply(lambda x: True if (x['UnconditionalLabel'] == True)
                          or (x['ConditionalLabel'] == True) else False,axis=1)
temp = data[data.check==True].reset_index(drop=True)
print(len(temp))
temp2 = data[data.check ==False].reset_index(drop=True).sample(frac= 0.3, random_state=42)
print(len(temp2))
df = pd.concat([temp,temp2], ignore_index=True)


3056
2845


In [8]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 5901
0 . ConditionalLabel : 1986
1 . UnconditionalLabel : 1312

 Classes Representation in Test Dataset: 1190
0 . ConditionalLabel : 192
1 . UnconditionalLabel : 136


## Model

In [9]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[codecarbon INFO @ 15:19:58] Energy consumed for RAM : 0.000158 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 15:19:58] Energy consumed for all GPUs : 0.000398 kWh. Total GPU Power : 12.018702366110706 W
[codecarbon INFO @ 15:19:58] Energy consumed for all CPUs : 0.001417 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:19:58] 0.001974 kWh of electricity used since the beginning.


In [10]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Training data: 5901
Validation data: 1190


Map:   0%|          | 0/5901 [00:00<?, ? examples/s]

Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

Map:   0%|          | 0/5901 [00:00<?, ? examples/s]

Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

## Metrics

In [11]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [12]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "cosine",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

# Training

In [13]:
multi_trainer =  Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

[codecarbon INFO @ 15:20:13] [setup] RAM Tracking...
[codecarbon INFO @ 15:20:13] [setup] GPU Tracking...
[codecarbon INFO @ 15:20:13] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:20:13] [setup] CPU Tracking...
[codecarbon WARNING @ 15:20:13] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:20:13] Energy consumed for RAM : 0.000178 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 15:20:13] Energy consumed for all GPUs : 0.000517 kWh. Total GPU Power : 28.67046230019357 W
[codecarbon INFO @ 15:20:13] Energy consumed for all CPUs : 0.001594 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:20:13] 0.002290 kWh of electricity used since the beginning.
[codecarbon WARNING @ 15:20:14] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 15:20:14] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 15:20:14] >>> Tracker's metadata:
[codecarbon

In [ ]:
multi_trainer.train()

In [15]:
tracker.stop()

[codecarbon INFO @ 15:43:17] Energy consumed for RAM : 0.002003 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 15:43:17] Energy consumed for all GPUs : 0.026983 kWh. Total GPU Power : 38.717416611714924 W
[codecarbon INFO @ 15:43:17] Energy consumed for all CPUs : 0.017928 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:43:17] 0.046913 kWh of electricity used since the beginning.


0.01823528936459293

In [16]:
multi_trainer.push_to_hub()

events.out.tfevents.1709652014.5e87334befb5.429.0:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ppsingh/CONDITIONAL-multilabel-climatebert/commit/f992547271bf9e61c181cb8206033544187ef748', commit_message='End of training', commit_description='', oid='f992547271bf9e61c181cb8206033544187ef748', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
log_results = multi_trainer.state.log_history.copy()
logs = []
while len(log_results)>1:

    a = log_results[0]
    b = log_results[1]
    logs.append({**a,**b})
    log_results.pop(0)
    log_results.pop(0)

results_df = pd.DataFrame(logs)
results_df

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_precision-micro,eval_precision-samples,eval_precision-weighted,eval_recall-micro,eval_recall-samples,eval_recall-weighted,eval_f1-micro,eval_f1-samples,eval_f1-weighted,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.5644,2.561554,0.000060,1.0,369,0.416085,0.364173,0.139076,0.416745,0.564024,0.141597,0.564024,0.442584,0.138936,0.437212,14.7777,80.527,5.075
1,0.4290,4.720606,0.000053,2.0,738,0.361624,0.442043,0.180252,0.479373,0.685976,0.176891,0.685976,0.537634,0.176751,0.547340,14.8452,80.160,5.052
2,0.2657,4.922025,0.000037,3.0,1107,0.423328,0.412621,0.194958,0.422925,0.777439,0.198739,0.777439,0.539112,0.194398,0.541801,14.8112,80.344,5.064
3,0.1482,7.004785,0.000020,4.0,1476,0.430143,0.490982,0.189076,0.494402,0.746951,0.190756,0.746951,0.592503,0.188235,0.592450,14.9383,79.661,5.021
4,0.0690,3.010533,0.000005,5.0,1845,0.501643,0.512605,0.192017,0.519310,0.743902,0.191176,0.743902,0.606965,0.189916,0.608983,15.0761,78.933,4.975
5,0.0353,0.067874,0.000000,6.0,2214,0.546037,0.502033,0.195378,0.504724,0.753049,0.193697,0.753049,0.602439,0.192717,0.603341,14.9233,79.741,5.026


# Predict and Evaluate

## Trainer

In [19]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>thresh).bool()
y_true = y_true.bool()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [20]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ConditionalLabel
[[837 161]
 [ 45 147]] 

UnconditionalLabel
[[970  84]
 [ 36 100]] 



In [21]:
# classw eights
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ConditionalLabel,0.477273,0.765625,0.588000,192.0
1,UnconditionalLabel,0.543478,0.735294,0.625000,136.0
2,micro avg,0.502033,0.753049,0.602439,328.0
3,macro avg,0.510375,0.750460,0.606500,328.0
4,weighted avg,0.504724,0.753049,0.603341,328.0
5,samples avg,0.195378,0.193697,0.192717,328.0
